# What is this?
A simple animated visualization of DC Fast Chargers that have 4 or more dispensers. 
This runs very slow so that points get added to the map and don't just flash for each date.

# How to use
* Run the Download_Latest_Data notebook first.
* Read the How to use in that notebook.
* Profit!

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go  
import geopandas as gpd

In [ ]:
df = pd.read_csv ('latest.csv', low_memory=False)

In [ ]:
df.dropna(subset=["Open Date"], inplace=True)
df = df[df['EV DC Fast Count'] > 3]

In [ ]:
# This is REALLY slow
df['dttime'] = pd.to_datetime(df['Open Date'], format='%Y-%m-%d')

df = df.sort_values(by='dttime')

# Add a column for animation frame
df['Frame'] = df['dttime'].dt.strftime('%Y-%m')

# Expand the dataframe to include all previous events up to each frame
expanded_df = pd.DataFrame()

for date in df['dttime'].unique():
    temp_df = df[df['dttime'] <= date].copy()
    temp_df['Frame'] = date.strftime('%Y-%m')
    expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)

In [ ]:
fig = px.scatter_geo(expanded_df,
                     height=1000,
                     width=1800,
                     lat=expanded_df.Latitude,
                     lon=expanded_df.Longitude,
                     animation_frame=expanded_df['Frame'],
                     range_color=(0, 100),
                     hover_name=expanded_df["Station Name"],
                     hover_data=[expanded_df['EV Network'], expanded_df.City, expanded_df.State],
                     size=expanded_df['EV DC Fast Count'],
                     color=expanded_df['EV DC Fast Count'],
                     scope = "usa"
                    )

# Keep all markers visible for the entire animation
fig.update_traces(marker=dict(size=len(expanded_df)))
fig.update_geos(
    landcolor="LightGreen",
    showocean=True,
    oceancolor="LightBlue",
    projection_type="albers usa"
)

fig.show()